In [1]:
import sys
sys.path.append("..")
from data.data_loader import cnn_data_loader_cv
from models.set_model import CNN_base

In [2]:
from models.CNN.D1D2Conv import Deep_D1D2_feature, D1D2_model
from models.CNN.BrainNetCNN import BrainNetCNN
from models.CNN.deep.StandardCNN import CNN

In [3]:
from glob import glob
from tqdm import tqdm

import torch
import networkx as nx

In [4]:
def model_parameter_counter(model):
    num_params = 0
    for p in model.parameters():
        if p.requires_grad:
            num_params += p.numel()
    return num_params

In [5]:
model = CNN(4, 100)
brain = BrainNetCNN(torch.ones(1,1,100,100), 4)

feature = Deep_D1D2_feature(100)
D1D2 = D1D2_model(feature, "complex", 4, 100)

In [6]:
print(model_parameter_counter(model))
print(model_parameter_counter(brain))
print(model_parameter_counter(D1D2))

407754
485339
396104


In [5]:
from data.data_loader import gnn_data_loader_cv

In [52]:
import torch
import networkx as nx
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from glob import glob
from tqdm import tqdm
from torch_geometric.utils import from_networkx
from torch_geometric.data.data import Data
# for gnn loader
def gnn_data_loader_cv(name):
    path = f"../graph_data/{name}/graph_tensor"
    data_list = []
    for i in range(20000):
        data = Data(
            x=torch.load(f"{path}/x/{i}.pt"),
            y=torch.load(f"{path}/y/{i}.pt"),
            edge_index=torch.load(f"{path}/edge/{i}.pt"),
        )
        data_list.append(data)

    return data_list, 4

In [59]:
data_list, num_class = gnn_data_loader_cv("new_poisson")

In [60]:
a = set()
for i in range(20000):
    a.add(data_list[i].y.item())

In [61]:
a

{0, 1, 2, 3}

In [45]:
import os
from tqdm import tqdm

In [48]:

for name in ["poisson", "new_poisson"]:
    data_list, num_class = gnn_data_loader_cv(name)    
    save_dir = f"../graph_data/{name}/graph_tensor"

    os.makedirs(f"{save_dir}/edge", exist_ok=True)
    os.makedirs(f"{save_dir}/x", exist_ok=True)
    os.makedirs(f"{save_dir}/y", exist_ok=True)
    print("="*20)
    print("save")
    for i, d in enumerate(tqdm(data_list)):
        torch.save(d.edge_index, f"{save_dir}/edge/{i}.pt")
        torch.save(d.x, f"{save_dir}/x/{i}.pt")    
        torch.save(d.y, f"{save_dir}/y/{i}.pt")    

  2%|▏         | 377/20000 [00:00<00:05, 3760.80it/s]

save


  2%|▏         | 382/20000 [00:00<00:05, 3817.50it/s]

save


100%|██████████| 20000/20000 [00:05<00:00, 3873.10it/s]


In [47]:
print("="*20)
print("save")
for i, d in enumerate(tqdm(data_list)):
    torch.save(d.edge_index, f"{save_dir}/edge/{i}.pt")
    torch.save(d.x, f"{save_dir}/x/{i}.pt")    
    torch.save(d.y, f"{save_dir}/y/{i}.pt")

  2%|▏         | 393/20000 [00:00<00:04, 3927.15it/s]

save


100%|██████████| 20000/20000 [00:05<00:00, 3979.17it/s]


In [43]:
from torch_geometric.data import Data

Data(x=torch.load(f"../graph_data/{i}.pt"))

Data(edge_index=[2, 2112], num_nodes=100, x=[100, 1], y=[1])

In [49]:
data_list[0].num_nodes

76

In [23]:
k = data_list[0].edge_index

In [33]:
torch.tensor([k, data_list[1].edge_index]).shape

TypeError: only integer tensors of a single element can be converted to an index

In [41]:
torch.save(data_list[0].edge_index, "tensor.pt")

In [42]:
torch.load("tensor.pt")

tensor([[ 0,  0,  0,  ..., 99, 99, 99],
        [ 1,  2,  3,  ..., 72, 76, 86]])

In [40]:
with open('data.json', 'w') as fp:
    json.dump(data_list[0].to_dict(), fp)



TypeError: Object of type Tensor is not JSON serializable